In [1]:
# Import necessary libraries
from pynq import Overlay, allocate
import numpy as np
import timeit

In [2]:
# Load the overlay
overlay = Overlay('/home/xilinx/jupyter_notebooks/overlays/pynq_ge_xsa.xsa')

In [3]:
A = None
B = None
X = None

In [4]:
# Allocate memory for the input and output buffers
n = 40  # size of your system
A = allocate(shape=(n, n), dtype=np.float32)
B = allocate(shape=(n,), dtype=np.float32)
X = allocate(shape=(n,), dtype=np.float32)


In [5]:
#Initialize input A and B
A[:] = np.random.rand(n, n).astype(np.float32)
B[:] = np.random.rand(n).astype(np.float32)

In [6]:
ip = overlay.solve_linear_equatio_0
#print(overlay.ip_dict["solve_linear_equatio_0"])

In [7]:

ip_metadata = overlay.ip_dict['solve_linear_equatio_0']

# Check the offsets for A, B, and X
offset_A = ip_metadata['registers']['A_1']['address_offset']
offset_B = ip_metadata['registers']['B_1']['address_offset']
offset_X = ip_metadata['registers']['X_1']['address_offset']
# Print the offsets
print("Offset for A:", hex(offset_A))
print("Offset for B:", hex(offset_B))
print("Offset for X:", hex(offset_X))


Offset for A: 0x18
Offset for B: 0x24
Offset for X: 0x30


In [8]:
def run_kernel():
    ip.write(0x10, n) 
    ip.write(0x18, A.physical_address)
    ip.write(0x24, B.physical_address)  
    ip.write(0x30, X.physical_address) 
    
    ip.write(0x00, 0x1)  # start the IP
    while (ip.read(0x00) & 0x2) == 0x0:  # wait for the done bit to be set
        pass
    
    return np.copy(X)

result = run_kernel()
print(result)

[-0.3173108  -0.5308561  -0.18501948  0.69623286  1.0620853   0.26061848
 -0.25652105 -0.5799922  -0.7429577   1.4791265  -0.60371244 -0.17431477
  0.1690831  -0.00311804 -0.40161338 -0.35442713 -0.39086536  0.66906005
 -0.47539723 -1.0945956   0.07488815 -0.7555655  -0.7828874  -0.88660586
 -0.65403825  1.4754039  -0.6660709  -0.19028868  1.9427922   0.6995583
 -0.5350745   0.23272324 -0.60170454 -1.063838    0.8086866   0.4367388
  0.7486617  -0.086447    1.1218308   1.2682514 ]


In [9]:
%%timeit 
run_kernel()

1.8 ms ± 6.96 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [10]:
%%timeit
np.linalg.solve(A, B)

913 µs ± 9.22 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
